In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('songdata.csv')
df.head(2)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."


In [4]:
df.shape

(57650, 4)

In [12]:
df = df.sample(n=5000).drop('link',axis=1).reset_index(drop=True)

Contant Based Recommendation system

cleaning

In [15]:
df['song'][0]

'A Kind Of Magic'

In [16]:
df['text'][0]

"It's a kind of magic  \nIt's a kind of magic  \nA kind of magic - no way  \nOne dream, one soul, one prize  \nOne goal, one golden glance of what should be  \nIt's a kind of magic  \nOne shaft of light that shows the way  \nNo mortal man can win this day  \nIt's a kind of magic  \nThe bell that rings inside your mind  \nIs challenging the doors of time  \nIt's a kind of magic  \nThe waiting seems eternity  \nThe day will dawn of sanity  \nOoh ooh ooh ooh  \nIs this a kind of magic?  \nIt's a kind of magic  \nThere can be only one  \nThis rage that lasts a thousand years  \nWill soon be done  \nThis flame that burns inside of me  \nI'm hearing secret harmonies  \nIt's a kind of magic  \nThe bell that rings inside your mind  \nIs challenging the doors of time  \nIt's a kind of magic  \nIt's a kind of magic  \nThis rage that lasts a thousand years  \nWill soon be, will soon be, will soon be done  \nThis is (this is) a kind (a kind) of magic (yeah)  \nThere can be only one one one one  \n

In [22]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ',regex=True)

In [23]:
df['text']

0       it's a kind of magic   it's a kind of magic   ...
1       na na na na na na na   na na na na na na na   ...
2       here i am, dancing in the wind   my back to yo...
3       when i look out over a green field of clover  ...
4       making somebody happy is a question of give an...
                              ...                        
4995    so fine   so fine, yeah!   and babe you're min...
4996    dying is easy   it's what i'm living for   i h...
4997    there's 52 ways to murder anyone   one and two...
4998    christmas celebration   all around the nation ...
4999    a man we passed just tried to stare me down   ...
Name: text, Length: 5000, dtype: object

loveing,loved,love  ==  love
(this is called stemming)

In [34]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.wordpunct_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [36]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [37]:
df['text']

0       it ' s a kind of magic it ' s a kind of magic ...
1       na na na na na na na na na na na na na na na n...
2       here i am , danc in the wind my back to your f...
3       when i look out over a green field of clover o...
4       make somebodi happi is a question of give and ...
                              ...                        
4995    so fine so fine , yeah ! and babe you ' re min...
4996    die is easi it ' s what i ' m live for i heard...
4997    there ' s 52 way to murder anyon one and two a...
4998    christma celebr all around the nation won ' t ...
4999    a man we pass just tri to stare me down and wh...
Name: text, Length: 5000, dtype: object

Cosine similerity

Tf stands for trem frequency

(how many times words repect)

Tf = (Word/Total Word)

idf = (Total Documents/Total Word)

word count = (Tf * idf)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [39]:
tfidf = TfidfVectorizer(stop_words='english')
matrix = tfidf.fit_transform(df['text'])

In [43]:
similarity = cosine_similarity(matrix)

In [45]:
similarity[0]

array([1.        , 0.00150267, 0.01564299, ..., 0.02114752, 0.01568068,
       0.00224462])

In [46]:
df['song'][2]

'Dancing'

In [47]:
df[df['song'] == 'Dancing']

,artist,song,text
2,Miley Cyrus,Dancing,"here i am , danc in the wind my back to your f..."
1485,John Martyn,Dancing,"oh darl , it ' s gonna be late when we get bac..."


In [74]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs


In [83]:
recommendation('Galaxy')

['I Just Fall In Love Again',
 'He Loves Me',
 "I'm Never Gonna Fall In Love Again",
 'Kiss Me',
 'All I Can Do',
 'And I Love Her',
 'Could This Be Love',
 'Moses',
 'Better Than Life',
 'Uh Oh',
 'Is This Love?',
 'I Believe (When I Fall In Love It Will Be Forever)',
 'Let It All Fall Down',
 'Nobody',
 'I Love A Lonely Day',
 'I Wish You Love',
 'I Will Still Love You',
 'No One Can Love You More',
 'Lonely Love',
 'Oh Where Oh Where Is Love?']

In [84]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))